In [ ]:
import os
import sys
import pandas as pd

import matplotlib.ticker as mtick
from matplotlib import rcParams
from matplotlib import pyplot as plt
from itertools import cycle



In [ ]:
file_list = ["C:/Users/User/Documents/BGI/newvolume/BGI/results/reference for stereopy/adult_brain/agglomerative n16 result/SS200000141TL_B5/SS200000141TL_B5.csv","C:/Users/User/Documents/BGI/newvolume/BGI/results/reference for stereopy/adult_brain/agglomerative n16 result/SS200000141TL_A4/SS200000141TL_A4.csv","C:/Users/User/Documents/BGI/newvolume/BGI/results/reference for stereopy/adult_brain/agglomerative n16 result/SS200000128TR_E2/SS200000128TR_E2.csv"]

df_list = []

for fpath in file_list:
    df_list.append(pd.read_csv(fpath))
    

In [ ]:
os.path.basename(file_list[0]).split('.')

In [ ]:
df_list[0]

In [ ]:
        fig, ax = plt.subplots(figsize=(20,15))
        fig.subplots_adjust(wspace=0)
        rcParams['figure.facecolor'] = 'white'
        rcParams['axes.facecolor'] = 'white'
        rcParams["figure.dpi"] = 200

        greys=cycle(['#1560BD', '#C5E17A', '#8D90A1'])
        names=['B5','A4','E2']
        colors = [next(greys) for _ in range(len(df_list))]
        cell_percentage_dfs = []
        plot_columns = []
        for id, df in enumerate(df_list):

            cell_percentage_dfs.append(df.loc[:, 'tissue_sliding_window'].value_counts(normalize=True).mul(100).rename(names[id]))
            plot_columns.append(names[id])
            if id==0:
                # cell_percentage_dfs[0] = cell_percentage_dfs[0].append(pd.Series([0.0], index=['unknown']))
                cell_percentage_dfs[0].index = cell_percentage_dfs[0].index.values.astype('str')

        cummulative_df = pd.concat(cell_percentage_dfs, axis=1).fillna(0)
        cummulative_df = cummulative_df.loc[sorted(cummulative_df.index.values, key=lambda x: float(x) if x != "unknown" else float('inf'))]
        cummulative_df.plot(y=plot_columns, kind="bar", rot=0, ax=ax, xlabel="", color=colors)

        ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
        ax.grid(False)
        ax.set_facecolor('white')
        plt.legend(loc='upper left', bbox_to_anchor=(1.04, 1))

        plt.savefig(os.path.join(os.path.dirname(file_list[0]), f'cluster_abundance_all_slices.png'), bbox_inches='tight')
        plt.show()
        plt.close()

In [ ]:
import sys
sys.path.append("stereo/algorithm/")

from stereo.core.stereo_exp_data import AnnBasedStereoExpData
from stereo.algorithm.community_detection import CommunityDetection

import pandas as pd
import os

Read csv data from one or several samples as anndata objects and place them in a list

In [ ]:
file_list = ["C:/Users/User/Documents/BGI/newvolume/BGI/results/reference for stereopy/adult_brain/agglomerative n16 result/SS200000141TL_B5/SS200000141TL_B5.h5ad","C:/Users/User/Documents/BGI/newvolume/BGI/results/reference for stereopy/adult_brain/agglomerative n16 result/SS200000141TL_A4/SS200000141TL_A4.h5ad","C:/Users/User/Documents/BGI/newvolume/BGI/results/reference for stereopy/adult_brain/agglomerative n16 result/SS200000128TR_E2/SS200000128TR_E2.h5ad"]
adata_list = []
for fpath in file_list: 
    adata_list.append(AnnBasedStereoExpData(fpath))

read already defined CCD annotation and add to adata

In [ ]:
df_list = []
for fpath in file_list:
    cur_df = pd.read_csv(os.path.join(os.path.dirname(fpath), os.path.basename(fpath).split('.')[0] + '.csv'))
    cur_df['tissue_sliding_window'] = cur_df['tissue_sliding_window'].astype('str')
    df_list.append(cur_df)

In [ ]:
for adata, labels in zip(adata_list, df_list):
    adata._ann_data.obs['tissue_sliding_window'] = 'unknown'
    adata._ann_data.obs.loc[labels.iloc[:,0].values, 'tissue_sliding_window'] = labels.loc[:, 'tissue_sliding_window'].values

In [ ]:
cd = CommunityDetection(slices=adata_list, annotation='celltype_pred', win_sizes='200', sliding_steps='50', cluster_algo='agglomerative', n_clusters=16, plotting=2, hide_plots=True)
cd.run_plots()